# Riemann solver tester


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#from ipywidgets import StaticInteract, RangeWidget
from ipywidgets import interact, interact_manual
import ipywidgets
from IPython.display import FileLink
from clawpack import riemann

The next cell imports a module containing a function that takes a Riemann problem (left state, right state, and approximate solver), and computes the Riemann solution, as well as functions to plot the solution in various forms.

In [ ]:
from clawpack.riemann import riemann_tools

# Acoustics: exact solution
We can use this to examine the exact solution of an acoustics Riemann problem.

This is a linear hyperbolic system of two equations for $q = [p, u]^T$, where $p$ is the pressure perturbation and $u$ is the velocity.  The system is $q_t + Aq_x = 0$, where the coefficient matrix is

$$
A = \left[\begin{array}{cc}0&K\\1/\rho&0\end{array}\right], 
$$

where $\rho$ is the density and $K$ the bulk modulus.  If we define the sound speed $c = \sqrt{K/\rho}$ and impedance $Z=\sqrt{K\rho}$, then the eigenvalues of the matrix are $s^1 = -c$ and $s^2 = +c$ and the corresponding eigenvectors are
$$
r^1 = \left[\begin{array}{c}-Z\\1\end{array}\right], \qquad r^2 = \left[\begin{array}{c}Z\\1\end{array}\right].
$$

For arbitrary states $q_\ell$ and $q_r$, the Riemann solution consists of two waves propagating with velocities $\pm c$ with an intermediate state $q_m$ that is connected to $q_\ell$ by a multiple of $r^1$ and to $q_r$ by a multiple of $r^2$.

This Riemann solver can be solved by the PyClaw solver `riemann.acoustics_1D_py.acoustics_1D`:

In [ ]:
solver = riemann.acoustics_1D_py.acoustics_1D
num_eqn = riemann.acoustics_1D_py.num_eqn

q_l = np.array((1,4))
q_r = np.array((3,7))

problem_data = {}
problem_data['zz'] = 2.0  # impedance 
problem_data['cc'] = 1.0  # sound speed

states, s, riemann_eval = riemann_tools.riemann_solution(solver,q_l,q_r,problem_data=problem_data)
riemann_tools.plot_phase(states)

Plot the solution at one particular time:

In [ ]:
fig = riemann_tools.plot_riemann(states,s,riemann_eval,0.5)

Plot over a range of times with a slider to control t:

In [ ]:
plot_function = riemann_tools.make_plot_function(states,s,riemann_eval)
widget = ipywidgets.IntSlider(min=0,max=len(figs)-1, value=0)
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))

# Euler equations: exact solution

We can compute the exact solution to the Riemann problem for the Euler equations.

In [ ]:
import Euler_exact_Riemann_solver
FileLink('Euler_exact_Riemann_solver.py')  # Link to examine the exact Riemann solver

In [ ]:
gamma = 1.4

q_l = np.array((3.,-0.5,2.))
q_r = np.array((1.,0.,1.))

ex_states, ex_speeds, reval = Euler_exact_Riemann_solver.exact_riemann_solution(q_l ,q_r, gamma)

plot_function = riemann_tools.make_plot_function(ex_states, ex_speeds, reval)
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))

# Euler: approximate solutions
We can also easily compare the approximate solutions given by, say, a Roe solver and an HLLE solver for the Euler equations.

## Roe solver:

In [ ]:
solver = riemann.euler_1D_py.euler_roe_1D
num_eqn = riemann.euler_1D_py.num_eqn

problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0
problem_data['efix'] = False

print "Roe solver solution to Euler equations:"
states, s, roe_eval = riemann_tools.riemann_solution(num_eqn,solver,q_l,q_r,problem_data=problem_data)
fig, ax = plt.subplots(1,2,figsize=(10,4))
riemann_tools.plot_phase(states,0,1,ax[0])
riemann_tools.plot_phase(states,0,2,ax[1])
riemann_tools.plot_phase_3d(states)

In [ ]:
plot_function = riemann_tools.make_plot_function(states,s,roe_eval)
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))

## HLLE Solver:

The HLLE solver uses only two waves with a constant state between that is uniquely defined by conservation for any choice of the two wave speeds.  The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characterstic speed $\lambda^3$ in the right state $q_r$.

In [ ]:
solver = riemann.euler_1D_py.euler_hll_1D
print "HLL solver solution to Euler equations:"
states_hll, s_hll, hll_eval = riemann_tools.riemann_solution(num_eqn,solver,q_l,q_r,problem_data=problem_data)
fig, ax = plt.subplots(1,2,figsize=(10,4))
riemann_tools.plot_phase(states_hll,0,1,ax[0])
riemann_tools.plot_phase(states_hll,0,2,ax[1])
riemann_tools.plot_phase_3d(states_hll)

In [ ]:
plot_function = riemann_tools.make_plot_function(states_hll,s_hll, hll_eval)
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))

## Compare the two approximate solvers:

In the plots below, red is the Roe solution, blue is the HLLE solution.

In [ ]:
plot_function = riemann_tools.make_plot_function([states_hll,states],[s_hll,s],[hll_eval,roe_eval])
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))

### Compare with the exact solution

In [ ]:
plot_function = riemann_tools.make_plot_function([ex_states,states_hll,states],
                                                 [ex_speeds,s_hll,s],
                                                 [reval,hll_eval,roe_eval],
                                                 ['Exact','HLLE','Roe'])
StaticInteract(plot_function, t=RangeWidget(0,.9,.1))